In [17]:
import sys
import numpy as np
import pandas as pd
import functools
import winsound
from Functions import datapath, beep

print('system: ', sys.version)
print('numpy: ', np.__version__)
print('panda: ', pd.__version__)

system:  3.5.1 |Anaconda 2.4.1 (64-bit)| (default, Dec  7 2015, 15:00:12) [MSC v.1900 64 bit (AMD64)]
numpy:  1.10.1
panda:  0.17.1


In [2]:
print('Reading Price data')
price_data = pd.read_pickle(datapath + 'history_M_Indicator.pickle')
price_data['DOCS25_sft1'] = price_data['DOCS(25)'].shift(1)
price_data['DOCR25_sft1'] = price_data['DOCR(25)'].shift(1)
price_data.columns

Reading Price data


Index(['DateTime', 'Seconds', 'Date', 'Time', 'Open', 'High', 'Low', 'Close',
       'Ticks', 'Close-Roll_Mean',
       ...
       'RSI(14)-Roll_Mean', 'RSI(14)-Roll_std', 'RSI(14)-Roll_var',
       'RSI(30)-Roll_Mean', 'RSI(30)-Roll_std', 'RSI(30)-Roll_var',
       'Break_Support', 'Break_Resistance', 'DOCS25_sft1', 'DOCR25_sft1'],
      dtype='object', length=102)

In [12]:
br_test = price_data.head(10000).copy()

In [18]:
i = np.where(br_test.Break_Resistance == True)[0]
print(i)
print(type(i))
if 58 in i:
    print ('yes')
    

[46 47 48 49 52 55 56 57 58 59 60 61 62 63 64 65 80 81]
<class 'numpy.ndarray'>
yes


In [26]:
i = np.delete(i, np.where(i <48))
i

array([48, 49, 52, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 80, 81], dtype=int64)

In [19]:
def beep(sound):
    winsound.PlaySound('%s.wav' % sound, winsound.SND_FILENAME)

In [4]:
def check_profit_loss_SELL_BR(df, BR, cur_ind):
    sl = 50
    tp = 50
    point = 0.0001
    xecn = 10
    
    if not BR:
        SELL = 'NO ORDER'  # if row has not broken resistance then no order, SELL will not be blank and skips while loop
    else:
        SELL = ''
    while (SELL == ''):
        ind = np.where(df.Break_Resistance == True)    # Get index of rows where high has broken resistance
        ind = np.delete(ind, np.where(ind < cur_ind))  # delete index if less than index of current row
        if SELL == '':
            for i in np.nditer(ind):   # for each index in list
                subdf = df.loc[int(i):]  # returns sub-dataframe from index to end of history data
                
                sl_sell = subdf.High.iloc[0] + (sl * point * xecn)   # stop loss from first index high price
                
                # Break resistance, SELL check for profit
                p = np.where(subdf.Low <= subdf.DOCS25_sft1)[0]  # list of index where low < support line
                # Break resistance, SELL check for loss
                l = np.where(subdf.High > sl_sell)[0]  # list of index where high > stop loss for sell
                #print(cur_ind,':', p.size, l.size)
                if (p.size > 0) & (l.size > 0):      # if both p and l have length
                    if p.min() < l.min():            # profit if p has smaller index
                        SELL = 'PROFIT'
                        break                        # once SELL value found, break out of for loop
                    if l.min() < p.min():            # loss if l has smaller index
                        SELL = 'LOSS'
                        break                        # once SELL value found, break out of for loop
                elif (p.size > 0) & (l.size == 0):   # if l has no size but p does must be profit
                    SELL = 'PROFIT'
                    break                            # once SELL value found, break out of for loop
                elif (p.size == 0) & (l.size > 0):   # if p has no size but l does must be loss
                    SELL = 'LOSS'
                    break                            # once SELL value found, break out of for loop
                elif (p.size ==0 ) & (l.size == 0):  # if neither have length than unk
                    SELL = 'UNK'
                    break                            # once SELL value found, break out of for loop
            #print(cur_ind, ': SELL=', SELL)
    return SELL

In [10]:
def check_profit_loss_BUY_BS(df, BS, cur_ind):
    sl = 50
    tp = 50
    point = 0.0001
    xecn = 10
    
    if not BS:
        BUY = 'NO ORDER' # if row has not broken support then no order, SELL will not be blank and skips while loop
    else:
        BUY = ''
    while (BUY == ''):
        ind = np.where(df.Break_Support == True)    # Get index of rows where low has broken resistance
        ind = np.delete(ind, np.where(ind < cur_ind))  # delete index if less than index of current row
        if BUY == '':
            for i in np.nditer(ind):   # for each index in list
                subdf = df.loc[int(i):]  # returns sub-dataframe from index to end of history data
                
                sl_buy = subdf.Low.iloc[0] - (sl * point * xecn)   # stop loss from first index low price
                
                # Break support, BUY check for profit
                p = np.where(subdf.High >= subdf.DOCR25_sft1)[0]  # list of index where High > resistance line
                # Break support, BUY check for loss
                l = np.where(subdf.Low < sl_buy)[0]  # list of index where low < stop loss for buy
                #print(cur_ind,':', p.size, l.size)
                if (p.size > 0) & (l.size > 0):      # if both p and l have length
                    if p.min() < l.min():            # profit if p has smaller index
                        BUY = 'PROFIT'
                        break                        # once BUY value found, break out of for loop
                    if l.min() < p.min():            # loss if l has smaller index
                        BUY = 'LOSS'
                        break                        # once BUY value found, break out of for loop
                elif (p.size > 0) & (l.size == 0):   # if l has no size but p does must be profit
                    BUY = 'PROFIT'
                    break                            # once BUY value found, break out of for loop
                elif (p.size == 0) & (l.size > 0):   # if p has no size but l does must be loss
                    BUY = 'LOSS'
                    break                            # once BUY value found, break out of for loop
                elif (p.size ==0 ) & (l.size == 0):  # if neither have length than unk
                    BUY = 'UNK'
                    break                            # once BUY value found, break out of for loop
            #print(cur_ind, ': BUY=', BUY)
    return BUY

In [13]:
br_test['BS_BUY'] = list(map(functools.partial(check_profit_loss_BUY_BS, br_test), br_test.Break_Support, br_test.index))

In [99]:
def check_profit_loss_BUY_BR(df, BR, cur_ind):
    sl = 50
    tp = 50
    point = 0.0001
    xecn = 10

    if not BR:
        BUY = 'NO ORDER' # if row has not broken resistance then no order, SELL will not be blank and skips while loop
    else:
        BUY = ''
    while (BUY == ''):
        ind = np.where(df.Break_Resistance == True)    # Get index of rows where high has broken resistance        
        ind = np.delete(ind, np.where(ind < cur_ind))  # delete index if less than index of current row
        if BUY == '':
            for i in np.nditer(ind):   # for each index in list
                subdf = df.loc[int(i):]  # returns sub-dataframe from index to end of history data

                tp_buy = subdf.High.iloc[0] + (tp * point * xecn)  # take profit from first index high price 
                sl_buy = subdf.High.iloc[0] - (sl * point * xecn)  # stop loss from first index high price
                #print(cur_ind, ':', tp_buy, sl_buy)
                # Break resistance, BUY check for profit
                p = np.where(subdf.High >= tp_buy)[0]  # list of index where high > take profit for buy
                # Break resistance, BUY check for loss
                l = np.where(subdf.Low <= sl_buy)[0]   # list of index where low < stop loss for buy
                #print('-- :', subdf.High.max(), subdf.Low.min())
                #print('--', p.size, l.size)
                #print()
                if (p.size > 0) & (l.size > 0):      # if both p and l have length
                    if p.min() < l.min():            # profit if p has smaller index
                        BUY = 'PROFIT'
                        break                        # once BUY value found, break out of for loop
                    if l.min() < p.min():            # loss if l has smaller index
                        BUY = 'LOSS'
                        break                        # once BUY value found, break out of for loop
                elif (p.size > 0) & (l.size == 0):   # if l has no size but p does must be profit
                    BUY = 'PROFIT'
                    break                            # once BUY value found, break out of for loop
                elif (p.size == 0) & (l.size > 0):   # if p has no size but l does must be loss
                    BUY = 'LOSS'
                    break                            # once BUY value found, break out of for loop
                elif (p.size ==0 ) & (l.size == 0):  # if neither have length than unk
                    BUY = 'UNK'
                    break                            # once BUY value found, break out of for loop
            #print(cur_ind, ': BUY=', BUY)   
    return BUY

In [84]:
br_test['BR_BUY'] = list(map(functools.partial(check_profit_loss_BUY_BR, br_test), br_test.Break_Resistance, br_test.index))

In [67]:
br_test.groupby('BR_SELL').count()

,DateTime,Seconds,Date,Time,Open,High,Low,Close,Ticks,Close-Roll_Mean,...,RSI(14)-Roll_Mean,RSI(14)-Roll_std,RSI(14)-Roll_var,RSI(30)-Roll_Mean,RSI(30)-Roll_std,RSI(30)-Roll_var,Break_Support,Break_Resistance,DOCS25_sft1,BR_BUY
BR_SELL,,,,,,,,,,,,,,,,,,,,,
NO ORDER,8837,8837,8837,8837,8837,8837,8837,8837,8837,8823,...,8797,8797,8797,8797,8797,8797,8837,8837,8807,8837
PROFIT,1157,1157,1157,1157,1157,1157,1157,1157,1157,1157,...,1154,1154,1154,1154,1154,1154,1157,1157,1157,1157
UNK,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [68]:
br_test.groupby('BR_BUY').count()

,DateTime,Seconds,Date,Time,Open,High,Low,Close,Ticks,Close-Roll_Mean,...,RSI(14)-Roll_Mean,RSI(14)-Roll_std,RSI(14)-Roll_var,RSI(30)-Roll_Mean,RSI(30)-Roll_std,RSI(30)-Roll_var,Break_Support,Break_Resistance,DOCS25_sft1,BR_SELL
BR_BUY,,,,,,,,,,,,,,,,,,,,,
NO ORDER,8837,8837,8837,8837,8837,8837,8837,8837,8837,8823,...,8797,8797,8797,8797,8797,8797,8837,8837,8807,8837
UNK,1163,1163,1163,1163,1163,1163,1163,1163,1163,1163,...,1160,1160,1160,1160,1160,1160,1163,1163,1163,1163


In [85]:
price_data['BR_SELL'] = list(map(functools.partial(check_profit_loss_SELL_BR, price_data), price_data.Break_Resistance, price_data.index))
beep(sys.argv[1])

In [95]:
price_data['BR_BUY'] = list(map(functools.partial(check_profit_loss_BUY_BR, price_data), price_data.Break_Resistance, price_data.index))
beep(sys.argv[1])

In [ ]:
price_data['BS_BUY'] = list(map(functools.partial(check_profit_loss_BUY_BS, price_data), price_data.Break_Support, price_data.index))
beep(sys.argv[1])

In [96]:
price_data.groupby('BR_SELL').count()

,DateTime,Seconds,Date,Time,Open,High,Low,Close,Ticks,Close-Roll_Mean,...,RSI(14)-Roll_Mean,RSI(14)-Roll_std,RSI(14)-Roll_var,RSI(30)-Roll_Mean,RSI(30)-Roll_std,RSI(30)-Roll_var,Break_Support,Break_Resistance,DOCS25_sft1,BR_BUY
BR_SELL,,,,,,,,,,,,,,,,,,,,,
NO ORDER,324130,324130,324130,324130,324130,324130,324130,324130,324130,324116,...,322247,322247,322247,322247,322247,322247,324130,324130,322223,324130
PROFIT,41958,41958,41958,41958,41958,41958,41958,41958,41958,41958,...,41922,41922,41922,41922,41922,41922,41958,41958,41958,41958
UNK,23,23,23,23,23,23,23,23,23,23,...,22,22,22,22,22,22,23,23,23,23


In [97]:
price_data.groupby('BR_BUY').count()

,DateTime,Seconds,Date,Time,Open,High,Low,Close,Ticks,Close-Roll_Mean,...,RSI(14)-Roll_Mean,RSI(14)-Roll_std,RSI(14)-Roll_var,RSI(30)-Roll_Mean,RSI(30)-Roll_std,RSI(30)-Roll_var,Break_Support,Break_Resistance,DOCS25_sft1,BR_SELL
BR_BUY,,,,,,,,,,,,,,,,,,,,,
LOSS,18737,18737,18737,18737,18737,18737,18737,18737,18737,18737,...,18722,18722,18722,18722,18722,18722,18737,18737,18737,18737
NO ORDER,324130,324130,324130,324130,324130,324130,324130,324130,324130,324116,...,322247,322247,322247,322247,322247,322247,324130,324130,322223,324130
PROFIT,15874,15874,15874,15874,15874,15874,15874,15874,15874,15874,...,15858,15858,15858,15858,15858,15858,15874,15874,15874,15874
UNK,7370,7370,7370,7370,7370,7370,7370,7370,7370,7370,...,7364,7364,7364,7364,7364,7364,7370,7370,7370,7370
